In [1]:
import sys 
sys.path.append("../")

from wave_circuit import *

Linearized Spectrum Plot

In [2]:
n = 10
N = 2 ** n

exact_spec = np.sqrt(-spectrum_P(N)) * np.sign(np.linspace(0, N-1, N) - N//2)
lin_spec = linear_spectrum_P(N)

np.save('../data/spec_plot.npy', np.array([{
    'exact':exact_spec, 
    'linear':lin_spec, 
    }]), allow_pickle=True)

Energy Plot

In [3]:
from qiskit.quantum_info import Operator
sigma = 0.1 
mu = 0.5

f = lambda x: 2/(np.sqrt(3 * sigma) * (np.pi)**(1/4)) * (1 - ((x-mu)/sigma)**2 ) * np.exp(- (x-mu)**2/(2*sigma**2))
df = lambda x: -4/(np.sqrt(3 * sigma) * (np.pi)**(1/4)) * (x-mu)/(sigma**2) * np.exp(- (x-mu)**2/(2*sigma**2)) - (x-mu)/(sigma**2) * f(x)


init_state = f(np.linspace(0,1-1/N,N) + 1/(2*N))
init_state = np.concatenate([init_state, np.zeros(N)])
K = np.linalg.norm(init_state)

init_state = init_state/K

f_extend = lambda x : f(x - np.floor(x))/K
df_extend = lambda x : df(x - np.floor(x))/K


f_evo = lambda x,t :  1/2 * (f_extend(x - t) + f_extend(x + t))

dxf_evo = lambda x,t :1/2 * (df_extend(x - t) + df_extend(x + t))
dtf_evo = lambda x,t : 1/2 * (-df_extend(x - t) + df_extend(x + t))


def linear_evo_op(n, t):
    qc = simulation_circuit(n, swap = False).assign_parameters([t])
    op = Operator(qc.reverse_bits()).data
    return op

def n_int(fun):
    return 1/N * np.sum([fun(_) for _ in np.linspace(1/(2*N), 1 - 1/(2*N), N)])

def T(t):
    return n_int(lambda x : dtf_evo(x,t)**2)

def U(t):
    return n_int(lambda x : dxf_evo(x,t)**2)

def circ_E(t):
    state = (linear_evo_op(n, t) @ init_state)
    T_state  = state[N:]
    U_state = state[:N]
    return (-U_state.conjugate().T @ (N**2 * L_P(N)) @ U_state / N).real, (-T_state.conjugate().T @ (N ** 2 * L_P(N)) @ T_state / N).real


data_dict = { 
             'exact_kin':[T(_) for _ in np.linspace(0, 1, 100)],
             'exact_pot':[U(_) for _ in np.linspace(0, 1, 100)],
             'circ':[circ_E(_) for _ in np.linspace(0, 1, 100)]
            }

np.save('../data/energies.npy', np.array([data_dict]), allow_pickle=True)


In [4]:
t_vals = np.linspace(0, 1, 100)
n_list = range(5, 11)
fid_full = []
for n in n_list:
    N = 2**n
    init_state = np.concatenate([f(np.linspace(1/(2*N), 1-1/(2*N), N)), np.zeros(N)])
    K = np.linalg.norm(init_state)
    init_state = init_state/K
    
    fid = []
    for t in t_vals:
        v1 = np.array([f_evo(_, t) for _ in np.linspace(1/(2*N), 1-1/(2*N), N)])
        v2 = (linear_evo_op(n, t) @ init_state)[:N]
        fid.append(np.max(abs(v1 - v2/(v2[np.argmax(abs(v2))]/max(abs(v2))))))
    fid_full.append(fid)

data_dict = { 
             'times':t_vals,
             'nlist':n_list,
             'fids':fid_full
            }

np.save('../data/evo_errors.npy', np.array([data_dict]), allow_pickle=True)